In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [31]:
def get_webpage(url, tag, attr_type, val): # 
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    if attr_type == "id":
        result = soup.find(tag, id=val)
    elif attr_type == "class":
        result = soup.find_all(tag, class_=val)
    return result
# get_webpage("https://movies.yahoo.com.tw/", "ul", "id", "mainmenu")

In [74]:
def get_movies_url(menuUrl):
    movieList = get_webpage(menuUrl, "div", "class", "release_movie_name")
    movieInfoList = list()
    for m in movieList:
        movieChiName = re.sub(r'\s', '', m.select("a")[0].text)
        movieEngName = re.sub(r'\s', '', m.select("a")[1].text)
        movieUrl = m.select("a")[0]['href']
        movieInfoList.append([movieChiName, movieEngName, movieUrl])
    return movieInfoList

In [180]:
def get_movies_info(movieInfo): # movieInfo: ['自殺突擊隊：集結', 'TheSuicideSquad', 'https://movies.yahoo.com.tw/movieinfo_main/%E8%87%AA%E6%AE%BA%E7%AA%81%E6%93%8A%E9%9A%8A-%E9%9B%86%E7%B5%90-the-suicide-squad-11328']
    moviePage = get_webpage(movieInfo[2], "div", "class", "movie_intro_info_r")[0]
    movieClassSec = moviePage.find("div", class_="level_name_box")
    movieClass = list()
    # 分類
    for info in movieClassSec.select("a"): # 篩選出分類
        movieClass.append(re.sub(r'\s', '', info.text))
        # print(info)
        pass
    movieInfo.append(movieClass)
    # 上映日期
    movieInfo.append(moviePage.find("span").text) # 上映日期：2021-08-05
    # 劇情介紹
    movieStory = get_webpage(movieInfo[2], "span", "id", "story")
    movieInfo.append(re.sub(r'\s', '', movieStory.text))

    return movieInfo[:2] + movieInfo[3:]
    # print(movieClass)
    # input()

In [181]:
def main():
    sel = get_webpage("https://movies.yahoo.com.tw/", "ul", "id", "mainmenu").select("a")
    mainmenu = dict()
    for s in sel:
        # print(s)
        for t in ['本週新片', '上映中', '即將上映']:
            if t in s.text: # 是以上種類的連結, 記錄在 mainmenu
                mainmenu[t] = s['href']
    # print(mainmenu)
    for movieName, movieUrl in mainmenu.items():
        movieList = get_movies_url(movieUrl)
        for m in movieList: # m: ['自殺突擊隊：集結', 'TheSuicideSquad', 'https://movies.yahoo.com.tw/movieinfo_main/%E8%87%AA%E6%AE%BA%E7%AA%81%E6%93%8A%E9%9A%8A-%E9%9B%86%E7%B5%90-the-suicide-squad-11328']
            print(get_movies_info(m))
main()

['自殺突擊隊：集結', 'TheSuicideSquad', ['動作', '冒險'], '上映日期：2021-08-05', '歡迎來到地獄—全美死亡率最高的「美夢監獄」。那裡關押著最兇惡的超級反派，他們為了逃離該處願意付出任何代價，甚至不惜加入超級神祕又黑幕重重的X特遣隊。這次九死一生的任務是什麼呢？首先結集「血腥運動」、「和平使者」、「迴旋鏢隊長」、「捕鼠人2」、「智者」、「鯊魚王」、「黑色守衛」、「標槍」，以及大家最喜歡的精神病患「哈莉奎茵」等一幫罪犯，給他們配上最精良的重裝武器，再把他們扔…請注意真的是用扔的，到與世隔絕又布滿敵軍的柯多瑪堤小島。他們徒步穿越每個角落都藏滿敵對兵力和武裝傭兵的叢林，突擊隊員在瑞克佛萊格上校一人隨行並帶領管束下，執行著搜索及摧毀的任務…阿曼達華勒的政府技工則從耳機裡提點監控，全程緊迫盯人。與往常一樣，他們只要踏錯一步就會屍骨無存（無論是死於敵方的手下、自己隊友的暗算，或是華勒的親自下令）。若真要下注賭個輸贏，明智的選擇絕對是押他們落敗，全軍覆沒。']
['驅魔禁區', 'Demonic', ['科幻', '恐怖', '懸疑/驚悚'], '上映日期：2021-08-05', '★《第九禁區》《極樂世界》導演全新超自然科幻懼作★科幻視效結合驅魔題材，宛如《全面啟動》結合《厲陰宅》！★有些惡行，是惡魔逼人做的……一名年輕女子意外喚醒滿懷惡意的駭人惡靈。現在，這股超自然力量將殘酷地揭露一對母女之間數十年感情裂痕的恐怖根源……。']
['路卡的夏天', 'LUCA', ['動畫', '冒險'], '上映日期：2021-08-06', '皮克斯全新原創《路卡的夏天》，故事背景發生在美麗的義大利濱海小鎮里維埃拉，一位名叫路卡的海怪男孩的難忘夏天，他與新摯友們在夏天經歷的無數冒險，包含他們在島上參加的競賽，但最刺激的不是競賽本身，而是隱藏他們呼之欲出的海怪身份…《路卡的夏天》是一個成長的故事，海怪男孩是曾經的你我，好奇心驅使你走出舒適圈，遇見真誠的友誼，學習鼓起勇氣由衷生活，完成自以為不可能的夢想…']
['薄荷糖', 'PeppermintCandy', ['劇情'], '上映日期：2021-08-06', '★韓國重量級名導李滄東「綠色三部曲」之一！★勇奪「南韓奧斯卡」大鐘獎最佳影片、導演、劇本、女配角、新人等五項大獎！★韓國影帝薛耿求

['詭老', 'Old', ['懸疑/驚悚'], '上映日期：2021-07-30', '獨具遠見的導演奈沙馬蘭將推出一部令人不寒而慄、疑雲重重的全新驚悚片，故事描述一個家庭前往熱帶地帶度假時，發現他們才剛在夏日海灘放鬆了幾個小時，竟然就迅速衰老，在一天內過完他們的一生。這部電影豪華的國際化卡司陣容包括金球獎得主蓋爾賈西亞貝納（亞馬遜影集《叢林中的莫札特》）、維琪克雷普（《霓裳魅影》）、魯夫斯席威爾（亞馬遜影集《高堡奇人》）、梁肯（《STARWARS：原力覺醒》）、妮基阿姆卡伯德（《朱比特崛起》）、艾碧李（HBO影集《逃出絕命村》）、亞倫皮爾（Syfy頻道影集《超人前傳》）、艾力克斯爾沃夫（《宿怨》）、安貝絲戴維茲（《龍紋身的女孩》）、艾莉莎斯坎倫（《她們》）、伊蒙艾略特（《STARWARS：原力覺醒》）、凱瑟琳查芬特（Showtime影集《婚外情事》）以及湯瑪遜麥肯錫（《兔嘲男孩》）。《詭老》一片由BlindingEdgePictures製作，奈沙馬蘭自編自製自導，他的劇本改編自皮耶奧斯卡李維及弗雷德里克彼得斯著作之圖像小說《沙堡》。這部電影的製片是艾希文拉傑（《異裂》、《分裂》）及馬克賓斯托克（《異裂》、AppleTV+影集《靈異女僕》），執行製片則是史蒂文施奈德。']
['罪之聲', 'TheVoiceofSin', ['劇情', '懸疑/驚悚'], '上映日期：2021-07-30', '★實力派演員小栗旬、「全能才子」星野源首度精彩共演★改編自作家塩田武士以日本昭和真實未破懸案「固力果．森永案」為題材的人氣推理小說★《現在，很想見你》導演土井裕泰×《法醫女王》編劇野木亞紀子打造懸疑震撼劇！★2020日本報知電影獎最佳作品獎、最佳男主角獎、最佳男配角獎★2020日刊體育電影大獎最佳作品獎、最佳男主角獎★2020日本電影金像獎最佳劇本獎、話題演員獎，入選優秀作品、優秀男主角獎、優秀男配角獎、優秀導演獎、優秀劇本獎等10項大獎★經過35年而甦醒的宿命……在平成時代將邁入尾聲之際，新聞記者阿久津英士（小栗旬飾）被選入昭和最大未解決事件的特別企畫組，追查30多年前的事件真相，阿久津為了找到所剩的證據，每天不停進行調查。在此事件中，犯罪集團在威脅錄音帶裡用了三名孩子的聲音，阿久津對此十分在意。另一方面，在京都經營西裝店的曾根俊也（星野源飾），在父親的遺物中發現了卡式錄

['寶貝老闆：家大業大', 'TheBossBaby:FamilyBusiness', ['動畫', '喜劇'], '上映日期：2021-08-20', '在夢工廠動畫入圍奧斯卡金像獎的賣座動畫片《寶貝老闆》續集電影《寶貝老闆：家大業大》一片中，天普頓兄弟－提姆（《X戰警》系列電影詹姆斯馬斯登配音）和他的寶貝老闆小弟弟泰德（亞歷鮑德溫配音）－早已長大成人並且漸行漸遠。提姆已經結婚生子，而且是一個家庭主夫，泰德則是一家避險基金公司的執行長。但是一個作風強勢、積極主動的全新寶貝老闆，即將把這對兄弟再湊在一起，並且啟發他們建立全新的家族事業。提姆和他負責賺錢養家的老婆卡蘿（伊娃朗格利亞配音），和他們超級聰明的七歲大女兒泰貝莎（《復仇者聯盟3：無限之戰》雅瑞安娜葛林布雷配音），以及萌翻天的新生寶貝女嬰蒂娜（Netflix動畫喜劇影集《馬男波傑克》艾米塞德麗絲配音）住在郊區。泰貝莎是A咖高級童年學習中心的資優生，她超崇拜她的泰德叔叔，更希望以後能跟他一樣當個霸氣老闆，但是提姆的內心卻仍然充滿過度活躍的童年想像力，而且擔心她會變成一個工作狂，反而會失去度過一個快樂童年的機會。當小寶寶蒂娜透露她是－登登！－寶貝公司的頂尖特工，正在執行一場任務，目的是揭發泰貝莎就讀的學校，以及其神秘創辦人爾文阿姆斯壯（傑夫高布倫配音）的黑暗內幕，這部續集電影就將以出乎意料的方式讓天普頓兄弟再度攜手合作，使得他們重新評估家庭的意義，並且發掘真正重要的事情。麗莎庫卓與吉米金莫則再度為泰德和提姆的父母配音。建立在第一部電影締造全球超過五億美元的亮眼票房成績之上，導演湯姆麥葛瑞斯再度執導《寶貝老闆：家大業大》這部續集電影，製片則是傑夫赫爾曼（《功夫熊貓3》）。']
